# Generator matrix example

To get started with 'Q' matrix, we begin by importing some useful python packages:

In this notebook we use the substitution exchangeblity matrix and get the equilibrium frequencies as input and returns the corresponding generator matrix Q.

### To get started with 'Q' matrix, we begin by importing some useful python packages:

In [2]:
# Python 3.9
# Q matrix generation for LG model
# Created by: Gholamhossein Jowkar <jowk@zhaw.ch>
# ACGT ZHAW
# Created date: Sep 2023
# Modified by:
# Modified date:

import sys, os, re
import numpy as np

# Defining length of amino acids elements:
aa_len = 20


***

# WAG substitution model

A   R   N   D   C   Q   E   G   H   I   L   K   M   F   P   S   T   W   Y   V

Ala Arg Asn Asp Cys Gln Glu Gly His Ile Leu Lys Met Phe Pro Ser Thr Trp Tyr Val

Symmetrical part of the rate matrix and aa frequencies, estimated from 3905 globular protein amino acid sequences forming 182 protein families.

The first part gives the amino acid 'equilibrium' frequencies (pi_i) estimated from the 3905 sequences.  

The second part indicates the symmetric 'exchangeability' parameters, where s_ij = s_ji.  The s_ij  are not scaled in the tex files but here we scale them to ensure that the rows of the matrix sum to zero.

And finally we compute the net replacement rate from i to j is Q_ij = s_ij*pi_j.

Prepared by Simon Whelan and Nick Goldman, September 2000.


#### Citation:
Whelan, S. and N. Goldman.  2001.  A general empirical model of protein evolution derived from multiple protein families using 	a maximum likelihood approach.  Molecular Biology and Evolution 18, 691-699.

***


### Declare three variables to store the information:

In [14]:
# Initialize an empty symmetric Q matrix
wag_q = np.zeros([aa_len, aa_len], dtype=float)

# Initialize an empty symmetric exchangeability matrix
wag_s = np.zeros([aa_len, aa_len], dtype=float)

# Initialize an empty background frequency matrix
wag_pi = np.zeros(aa_len, dtype=float)

### Having the equilibrium frequencies:

In [15]:
try:
    # Parsing the background probability:    
    with open('./samples/sub_model/wag_pi.dat') as back_file:
        
        for line in back_file:
            # Split the line into individual numbers 
            numbers = line.strip().split(" ")
            # Convert the numbers to floating-point numbers
            numbers = [float(num) for num in numbers]
    
    wag_pi = np.array(numbers)
    
    # Print the background frequency
    print("Background Frequency Matrix:")
    print(wag_pi)
            
except FileNotFoundError:
    print(f"File '{file_path}' not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")

Background Frequency Matrix:
[0.0866279 0.043972  0.0390894 0.0570451 0.0193078 0.0367281 0.0580589
 0.0832518 0.0244313 0.048466  0.086209  0.0620286 0.0195027 0.0384319
 0.0457631 0.0695179 0.0610127 0.0143859 0.0352742 0.0708956]


### Having the exchangeablity matrix:
We read an exchangeability text as input and returns the corresponding symmetric exchangeability matrix.

In [18]:
try:
    with open('./samples/sub_model/wag_ex.dat') as exchange_file:
        
        lst_element = []
        lst_element.append([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
        
        for line in exchange_file:
            
            # Split the line into individual numbers 
            numbers = line.strip().split(" ")
            
            # Convert the numbers to floating-point numbers
            numbers = [float(num) for num in numbers]
            
            # Update the maximum length if necessary
            max_length = max(aa_len, len(numbers))
            
            # Add the list of numbers to the lst_element list
            lst_element.append(numbers)

            # Pad the lists with zeros to make them all the same size
            for i, lst in enumerate(lst_element):
                while len(lst) < max_length:
                    lst.append(0)
                    
    # Convert the list of lists to a numpy array
    lower_triangle_matrix = np.array(lst_element)
    
    
    # Making a symmetric matrix from a lower triangle matrix:
    
    wag_s = lower_triangle_matrix + lower_triangle_matrix.T

    for i in range(aa_len):
        wag_s[i, i] = -np.sum(wag_s[i, :])  # Diagonal elements

    # Print the symmetric exchangeability matrix
    print("Symmetric Exchangeability Matrix:")
    print(wag_s)
            
except FileNotFoundError:
    print(f"File '{file_path}' not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")


Symmetric Exchangeability Matrix:
[[-1.89444120e+01  5.51571000e-01  5.09848000e-01  7.38998000e-01
   1.02704000e+00  9.08598000e-01  1.58285000e+00  1.41672000e+00
   3.16954000e-01  1.93335000e-01  3.97915000e-01  9.06265000e-01
   8.93496000e-01  2.10494000e-01  1.43855000e+00  3.37079000e+00
   2.12111000e+00  1.13133000e-01  2.40735000e-01  2.00601000e+00]
 [ 5.51571000e-01 -1.91362210e+01  6.35346000e-01  1.47304000e-01
   5.28191000e-01  3.03550000e+00  4.39157000e-01  5.84665000e-01
   2.13715000e+00  1.86979000e-01  4.97671000e-01  5.35142000e+00
   6.83162000e-01  1.02711000e-01  6.79489000e-01  1.22419000e+00
   5.54413000e-01  1.16392000e+00  3.81533000e-01  2.51849000e-01]
 [ 5.09848000e-01  6.35346000e-01 -2.59582488e+01  5.42942000e+00
   2.65256000e-01  1.54364000e+00  9.47198000e-01  1.12556000e+00
   3.95629000e+00  5.54236000e-01  1.31528000e-01  3.01201000e+00
   1.98221000e-01  9.61621000e-02  1.95081000e-01  3.97423000e+00
   2.03006000e+00  7.19167000e-02  1.086

### Converting an exchangeability matrix into a generator matrix Q:

The off-diagonal elements of Q are calculated by multiplying the equilibrium frequencies with the corresponding entries in the exchangebility matrix.

The diagonal elements of Q are calculated as the negative sum of the products of equilibrium frequencies and exchangebility matrix entries for the same character, excluding the diagonal element itself.

In [19]:
try:
    for i in range(aa_len):
            for j in range(aa_len):
                if i != j:
                    wag_q[i, j] = wag_pi[j] * wag_s[i, j]

    for i in range(aa_len):
        wag_q[i, i] = -np.sum([wag_pi[k] * wag_s[i, k] for k in range(aa_len) if k != i]) # Diagonal elements

    print("Generator Matrix Q:")
    print(wag_q)
            
except FileNotFoundError:
    print(f"File '{file_path}' not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")


Generator Matrix Q:
[[-1.06444471e+00  2.42536800e-02  1.99296524e-02  4.21562148e-02
   1.98298829e-02  3.33710782e-02  9.18985299e-02  1.17944490e-01
   7.74359826e-03  9.37017411e-03  3.43038542e-02  5.62143492e-02
   1.74255844e-02  8.08968436e-03  6.58325075e-02  2.34330242e-01
   1.29414648e-01  1.62752002e-03  8.49173454e-03  1.42217283e-01]
 [ 4.77814374e-02 -9.28350781e-01  2.48352939e-02  8.40297141e-03
   1.01982062e-02  1.11488148e-01  2.54969723e-02  4.86744136e-02
   5.22133528e-02  9.06212421e-03  4.29037192e-02  3.31941091e-01
   1.33235035e-02  3.94737888e-03  3.10955231e-02  8.51031180e-02
   3.38262340e-02  1.67440367e-02  1.34582713e-02  1.78549860e-02]
 [ 4.41670616e-02  2.79374343e-02 -1.38391348e+00  3.09721807e-01
   5.12150980e-03  5.66949643e-02  5.49932740e-02  9.37048960e-02
   9.66573079e-02  2.68616020e-02  1.13388974e-02  1.86830763e-01
   3.86584470e-03  3.69569221e-03  8.92751131e-03  2.76280124e-01
   1.23859442e-01  1.03458645e-03  3.83077812e-02  1.3

***
#### Comparing two Q matrices for WAG model:

In [20]:
WAG_from_parnk = np.array([-1.0644447077525, 0.024253680012, 0.0199296524112, 0.0421562148098, 0.019829882912, 0.0333710782038, 0.091898529865, 0.117944490096, 0.0077435982602, 0.00937017411, 0.034303854235, 0.056214349179, 0.0174255844392, 0.0080896843586, 0.065832507505, 0.234330242141, 0.129414648097, 0.0016275200247, 0.008491734537, 0.142217282556,
                         0.0477814374309, -0.9283507809291, 0.0248352939324, 0.0084029714104, 0.0101982061898, 0.11148814755, 0.0254969723473, 0.048674413647, 0.052213352795, 0.009062124214, 0.042903719239, 0.331941090612, 0.0133235035374, 0.0039473788809, 0.0310955230559, 0.085103118001, 0.0338262340451, 0.016744036728, 0.0134582713486, 0.0178549859644,
                         0.0441670615592, 0.027937434312, -1.38391347672512, 0.309721806842, 0.0051215097968, 0.056694964284, 0.0549932739622, 0.093704896008, 0.096657307877, 0.026861601976, 0.011338897352, 0.186830763486, 0.0038658446967, 0.00369569221099, 0.0089275113111, 0.276280123717, 0.123859441762, 0.00103458645453, 0.0383077812, 0.0139129779176,
                         0.0640178448442, 0.006477251488, 0.212232770148, -0.94297329251968, 0.00058492787022, 0.0226532677023, 0.358464938024, 0.0720614260512, 0.0227376245588, 0.001911353642, 0.0073109283823, 0.029764733853, 0.0020234831358, 0.00179593805976, 0.0194028221904, 0.074506504504, 0.0228715867982, 0.0018668150853, 0.0114891949562, 0.010799881226,
                         0.088970318416, 0.023225614652, 0.0103686978864, 0.00172817559999, -0.46436390434772, 0.00362939371299, 0.0012396736328, 0.0255311625132, 0.0060827096236, 0.00824576291, 0.033128997983, 0.00459221916954, 0.0076154533014, 0.015296664838, 0.0050066661924, 0.097857567114, 0.0312985388968, 0.010315697313, 0.0191832740086, 0.071047316584,
                         0.0787099366842, 0.133477006, 0.060339961416, 0.0351844479133, 0.00190795624962, -1.31468853172694, 0.317551411783, 0.0274774230936, 0.104910689643, 0.005521101322, 0.074957777201, 0.24159519414, 0.030136742202, 0.00384014619352, 0.0427139961732, 0.071524881773, 0.0523445036856, 0.0031035709083, 0.008032288082, 0.0213594972636,
                         0.137118971515, 0.019310611604, 0.0370254015012, 0.352205574616, 0.0004122601456, 0.200883241107, -1.17853838814971, 0.0472634621406, 0.0139264517825, 0.00617432607, 0.013298858967, 0.160308574698, 0.0061457688348, 0.00311812993141, 0.0312266801005, 0.0490058789081, 0.0501991141155, 0.0022522133463, 0.0069244312826, 0.0417384374836,
                         0.122727478488, 0.02570888938, 0.043997465064, 0.0493773258384, 0.0059212002572, 0.0121221828612, 0.0329610245313, -0.4741383934945, 0.006093410533, 0.0014757945466, 0.0052849306733, 0.0231712797588, 0.00339542007, 0.0019189431989, 0.011146518267, 0.093280508578, 0.0137786810791, 0.0048478037397, 0.0036545482168, 0.0132749884132,
                         0.0274570594166, 0.0939747598, 0.154649002326, 0.0530905054876, 0.0048071015816, 0.157714501491, 0.0330950244725, 0.020763831438, -0.9455247927498, 0.00669751654, 0.043058119558, 0.0552322503552, 0.0078818406807, 0.0261095183349, 0.0318601786938, 0.0514549945251, 0.0288777379989, 0.0037772913771, 0.136632497248, 0.0083910614248,
                         0.0167482050465, 0.008221840588, 0.0216647526984, 0.0022496876087, 0.003284932553, 0.0041839549677, 0.0073964135655, 0.00253502563518, 0.003376161347, -1.17498318692916, 0.27336615273, 0.0200868455952, 0.083031965142, 0.040717445093, 0.00457305166728, 0.022206797976, 0.088966278632, 0.0030567591897, 0.014821160614, 0.55449575628,
                         0.0344705408285, 0.021883589212, 0.0051413506032, 0.00483769259197, 0.0074197365386, 0.0319346789409, 0.0089563400907, 0.00510364337166, 0.0122025059606, 0.15368423202, -0.69175870029473, 0.015975776073, 0.094666495854, 0.081290001923, 0.0190303105564, 0.0239655313281, 0.0199280900994, 0.0095710687431, 0.0140609310556, 0.127636184504,
                         0.0785078337935, 0.23531264024, 0.117737663694, 0.0273733764605, 0.00142943173442, 0.14305227669, 0.150049162927, 0.0310993759044, 0.0217544113216, 0.015694841712, 0.022203558995, -1.07152398375532, 0.0182209045452, 0.0034141362684, 0.0254852873376, 0.067232846627, 0.084623394646, 0.0019781331795, 0.0047007809888, 0.0216539266904,
                         0.0774016821384, 0.030039999464, 0.0077483399574, 0.0059186573054, 0.0075393483596, 0.056754463806, 0.0182957528036, 0.01449413838, 0.0098736900133, 0.20634205636, 0.41846021018, 0.0579518322936, -1.2597234186325, 0.045758173097, 0.0078405461599, 0.0343352383995, 0.092502574724, 0.0074188949454, 0.0151127724254, 0.14593504782,
                         0.0182346531826, 0.004516408092, 0.00375891879174, 0.00266574034104, 0.007684890556, 0.00366990113448, 0.00471054498671, 0.0041568456258, 0.0165979167123, 0.05134827302, 0.18234669053, 0.0055103727096, 0.023220499701, -0.67999937105987, 0.0073881779164, 0.0379519766649, 0.0104882661681, 0.022005248076, 0.227669563576, 0.0460744832752,
                         0.124618565545, 0.029878490308, 0.0076255992414, 0.0241862096784, 0.0021123505512, 0.0342809801532, 0.0396167807095, 0.020277640926, 0.0170090221974, 0.0048431492208, 0.035849495396, 0.0345434792256, 0.0033413780883, 0.0062045996636, -0.5770185229931, 0.112151837712, 0.0485285253768, 0.0020054663895, 0.0076208498132, 0.0223241027972,
                         0.292004459041, 0.05383008268, 0.155350266162, 0.061138656376, 0.027178817748, 0.037788440247, 0.0409279829071, 0.111708930276, 0.0180832908897, 0.01548197904, 0.029719604451, 0.059989719918, 0.0096324810435, 0.0209811655989, 0.073828693968, -1.326554610767, 0.267114820854, 0.0075345000378, 0.0277605484806, 0.0165001710484,
                         0.183747304969, 0.024378648436, 0.079353827364, 0.0213842684566, 0.0099045924752, 0.0315100653768, 0.0477688308585, 0.0188010037494, 0.0115635053091, 0.07067118256, 0.028157755998, 0.086032427628, 0.029568433524, 0.0066065589057, 0.0363992375304, 0.304350756558, -1.1004826896859, 0.0015948784176, 0.0102700127816, 0.098419398788,
                         0.0098004742107, 0.05117989024, 0.00281118065298, 0.0074025714917, 0.013845044146, 0.0079236101097, 0.0090895272073, 0.0280544413194, 0.0064149020097, 0.010298201078, 0.057355623581, 0.008529242643, 0.0100576594062, 0.058786971516, 0.0063796049555, 0.0364094439818, 0.0067641119728, -0.44467569893618, 0.087670143938, 0.0259030544764,
                         0.0208543675065, 0.016776769076, 0.0424510884, 0.0185802165661, 0.0105002187974, 0.008363355651, 0.0113971362467, 0.0086252194872, 0.094633174672, 0.02036395922, 0.034364459162, 0.0082661793504, 0.0083556782799, 0.248050243532, 0.0098869347026, 0.0547101006747, 0.0177637255796, 0.035754572001, -0.6920103710931, 0.022312972188,
                         0.173776433679, 0.011074304228, 0.0076711383924, 0.0086899653085, 0.019349118692, 0.0110654786961, 0.0341810742559, 0.0155886497946, 0.0028916398054, 0.3790671258, 0.15520551106, 0.0189456434124, 0.040145332815, 0.0249765843548, 0.0144102052697, 0.0161795265281, 0.084699660521, 0.0052561618971, 0.011101848966, -1.034275403476])

# Reshape the 1D array into a 2D matrix with 20 row and 20 columns
new_wag = WAG_from_parnk.reshape(20, -1)

Is these two matrices: WAG from PRANK and the one we have compute are equal?

In [21]:
(np.round(wag_q, decimals=10)== np.round(new_wag, decimals=10)).all()

True

****